# 움직임 검출

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/swkim01/DSAC3/blob/main/gg-49-diff_camera.ipynb"><img src="https://github.com/swkim01/DSAC3/raw/main/colab_logo_32px.png" />구글 코랩에서 실행</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/swkim01/DSAC3/blob/main/gg-49-diff_camera.ipynb"><img src="https://github.com/swkim01/DSAC3/raw/main/GitHub-Mark-32px.png" />깃헙에서 소스 보기</a>
  </td>
</table>

### Camera로 부터 영상 재생 - Camera로부터 영상을 읽어, 화면에 보이기 위한 절차

- VideoCapture Object를 생성합니다. 변수로는 camera device index나 동영상 파일명을 
  넘겨줍니다. 일반적으로 0 이면 Camera와 연결이 됩니다.
- Loop를 돌면서 frame을 읽어 들입니다.
- 이전 연속된 두 프레임에서 변화된 부분을 추출하면 움직임을 추출한 후, 움직임이 발생한 부분(변화된 부분)을 화면에 보여줍니다.
- 영상 재생이 끝나면, VideoCapure Object를 release하고 window를 닫습니다.

- 동영상 재생시에는 해당 동영상의 Codec이 설치되어 있어야 합니다.

In [ ]:
import cv2
import numpy as np

In [ ]:
cap = cv2.VideoCapture(0)
img_last = None # 이전 프레임을 저장해둘 변수 --- (*1)
green = (0, 255, 0)

In [ ]:
while True:
    # 이미지 추출하기
    _, frame = cap.read()
    frame = cv2.resize(frame, (500, 300))
    # 흑백 이미지로 변환하기 --- (*2)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 0)
    img_b = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)[1]
    # 차이 확인하기
    if img_last is None:
        img_last = img_b
        continue
    frame_diff = cv2.absdiff(img_last, img_b) # --- (*3)
    cnts = cv2.findContours(frame_diff, 
            cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[0]
    # 차이가 있는 부분 출력하기 --- (*4)
    for pt in cnts:
        x, y, w, h = cv2.boundingRect(pt)
        if w < 50: continue # 작은 변경은 무시하기
        cv2.rectangle(frame, (x, y), (x+w, y+h), green, 2)
    # 프레임을 변수에 저장해두기 --- (*5)
    img_last = img_b
    # 화면에 출력하기
    cv2.imshow("Diff Camera", frame)
    cv2.imshow("diff data", frame_diff)
    if cv2.waitKey(1) == 13: break
    
cap.release()
cv2.destroyAllWindows()